# **Robustness of the algorithm**

In [21]:
# Tensorflow
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import legacy
from tensorflow.keras.backend import clear_session
from tensorflow.python.ops import rnn, rnn_cell

# Sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import precision_score, recall_score

import matplotlib.pyplot as plt
import seaborn as sns
import os
import ast
import pandas as pd
import numpy as np

# from google.colab import drive
# import import_ipynb

### Clear Previous Sessions

In [22]:
tf.keras.backend.clear_session()

In [23]:
%run Architectures.ipynb

#### Adding small noise to the original data and Generating predictions

##### Adversarial Attack: testing the model's resilience to small but intentional perturbations

In [24]:
def generate_adversarial_examples(X, model, epsilon=0.01):

    # Generate predictions for the original data
    original_predictions = model.predict(X)
    original_classes = np.argmax(original_predictions, axis=1)  #the classes predicted for the original data

    # Add small noise to the original data
    noise = np.random.normal(0, epsilon, X.shape)
    adversarial_X = X + noise   #the adversarially modified data

    # Ensure the data still falls within valid MFCC range
    adversarial_X = np.clip(adversarial_X, -1, 1)

    # Generate predictions for the adversarial data
    adversarial_predictions = model.predict(adversarial_X)
    adversarial_classes = np.argmax(adversarial_predictions, axis=1)    #the classes predicted for the adversarial data


    return adversarial_X, original_classes, adversarial_classes


### Test for a single iteration of the cross validation
##### fold1 - test
##### fold2 and fold3 - validation
##### fold3 - train

In [25]:
def robustness_test(nn_type, mfcc, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units):
    i = 1 #fold1 used for test

    # Define test set paths
    X_test_path = 'UrbanSound8K/audio/fold'+str(i)+'_4sec_mfccs_13/3D_array.npy'
    Y_test_path = 'UrbanSound8K/audio/fold'+str(i)+'_label/3D_array.npy'

    X_val_path1 = 'UrbanSound8K/audio/fold'+str((i+1-1) % 10 + 1)+f'_4sec_mfccs_{mfcc}/3D_array.npy'
    Y_val_path1 = 'UrbanSound8K/audio/fold'+str((i+1-1) % 10 + 1)+'_label/3D_array.npy'
    X_val_path2 = 'UrbanSound8K/audio/fold'+str((i+2-1) % 10 + 1)+f'_4sec_mfccs_{mfcc}/3D_array.npy'
    Y_val_path2 = 'UrbanSound8K/audio/fold'+str((i+2-1) % 10 + 1)+'_label/3D_array.npy'

    # Combine the validation paths
    X_val_paths = [X_val_path1, X_val_path2]
    Y_val_paths = [Y_val_path1, Y_val_path2]

    # # Define training set paths (all remaining folds)
    X_train_paths = ['UrbanSound8K/audio/fold'+str((j-1) % 10 + 1)+f'_4sec_mfccs_{mfcc}/3D_array.npy' for j in range(i+3, i+10)]
    Y_train_paths = ['UrbanSound8K/audio/fold'+str((j-1) % 10 + 1)+'_label/3D_array.npy' for j in range(i+3, i+10)]

    # Define test set paths
    X_test_path = f'UrbanSound8K/audio/fold{i}_4sec_mfccs_{mfcc}/3D_array.npy'
    Y_test_path = f'UrbanSound8K/audio/fold{i}_label/3D_array.npy'

    # Define validation set paths (wrapping around if i+2 > 10)
    X_val_path1 = f'UrbanSound8K/audio/fold{((i+1-1) % 10 + 1)}_4sec_mfccs_{mfcc}/3D_array.npy'
    Y_val_path1 = f'UrbanSound8K/audio/fold{((i+1-1) % 10 + 1)}_label/3D_array.npy'
    X_val_path2 = f'UrbanSound8K/audio/fold{((i+2-1) % 10 + 1)}_4sec_mfccs_{mfcc}/3D_array.npy'
    Y_val_path2 = f'UrbanSound8K/audio/fold{((i+2-1) % 10 + 1)}_label/3D_array.npy'

    # Combine the validation paths
    X_val_paths = [X_val_path1, X_val_path2]
    Y_val_paths = [Y_val_path1, Y_val_path2]

    # Define training set paths (all remaining folds)
    X_train_paths = [f'UrbanSound8K/audio/fold{((j-1) % 10 + 1)}_4sec_mfccs_{mfcc}/3D_array.npy' for j in range(i+3, i+11)]
    Y_train_paths = [f'UrbanSound8K/audio/fold{((j-1) % 10 + 1)}_label/3D_array.npy' for j in range(i+3, i+11)]

    # Load the datasets from the paths
    X_test = np.load(X_test_path)
    Y_test = np.load(Y_test_path)


    X_val1 = np.load(X_val_path1)
    X_val2 = np.load(X_val_path2)
    Y_val1 = np.load(Y_val_path1)
    Y_val2 = np.load(Y_val_path2)

    X_train = [np.load(path) for path in X_train_paths]
    Y_train = [np.load(path) for path in Y_train_paths]


    # Find the minimum size among all folds
    min_size = min([X_test.shape[0], X_val1.shape[0], X_val2.shape[0]] + [x.shape[0] for x in X_train])

    # Resize the data of each fold to the minimum size
    X_test_resized = X_test[:min_size]
    Y_test_resized = Y_test[:min_size]

    X_val1_resized = X_val1[:min_size]
    Y_val1_resized = Y_val1[:min_size]

    X_val2_resized = X_val2[:min_size]
    Y_val2_resized = Y_val2[:min_size]

    X_train_resized = [x[:min_size] for x in X_train]
    Y_train_resized = [y[:min_size] for y in Y_train]


    # Combine the validation sets
    X_test = X_test_resized
    Y_test = Y_test_resized
    X_val = np.concatenate((X_val1_resized, X_val2_resized), axis=2)
    Y_val = np.concatenate((Y_val1_resized, Y_val2_resized), axis=2)
    X_train = np.concatenate((X_train_resized[0], X_train_resized[1], X_train_resized[2], X_train_resized[3], X_train_resized[4], X_train_resized[5], X_train_resized[6]), axis=2)
    Y_train = np.concatenate((Y_train_resized[0], Y_train_resized[1], Y_train_resized[2], Y_train_resized[3], Y_train_resized[4], Y_train_resized[5], Y_train_resized[6]), axis=2)

    X_test = X_test.transpose(2,1,0)
    Y_test = Y_test.transpose(2,1,0)
    X_val = X_val.transpose(2,1,0)
    Y_val = Y_val.transpose(2,1,0)
    X_train = X_train.transpose(2,1,0)
    Y_train = Y_train.transpose(2,1,0)


#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    #Clear previous models
    clear_session()  

    # Call de neural network
    input_shape = (751, mfcc)
    num_classes = 10

    #Neural Network Initialization
    if nn_type == "mlp":
        model = build_mlp_model(input_shape, num_classes, l2_lambda, n_hidden_units)

    elif nn_type == "bd_rnn":
        model = build_bd_rnn_model(input_shape, num_classes, l2_lambda, n_hidden_units)

    else:
        print("Introduce a valid neural network")





    model.summary() # print The neural network's architecture

    # Define the optimizer
    optimizer = legacy.SGD(learning_rate=learning_rate, momentum=momentum)

    #output layer
    model.add(layers.Dense(10, activation='softmax'))

    
    Y_train = np.squeeze(Y_train, axis=1)
    Y_val = np.squeeze(Y_val, axis=1)



    # Compile the model.
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )


    history = model.fit(

        #training dataset
        X_train,
        Y_train,

        epochs=n_epochs,  # Number of epochs
        batch_size=batch_size, # Number of samples per batch

        #validation dataset
        validation_data=(X_val, Y_val)
    )

#----------------------------------------------------------------------------------------------------------------- Adding Robustness --------------------------------------------------------------------------------------------------------------------------
    
    # Select a batch of test data for generating adversarial examples
    batch_X_test = X_test[:batch_size]
    batch_Y_test = Y_test[:batch_size]

    print(batch_X_test.shape)
    print(batch_Y_test.shape)

    batch_Y_test = np.squeeze(batch_Y_test, axis=1)

    if batch_Y_test.ndim == 3:
        batch_Y_test = batch_Y_test[:, 0, :]

    print(batch_Y_test.shape)  

    # Generate adversarial examples
    adv_X_test, original_classes, adv_classes = generate_adversarial_examples(batch_X_test, model)

    # Evaluate the model on adversarial examples
    adv_scores = model.evaluate(adv_X_test, batch_Y_test, verbose=0)
    print(f"Adversarial Test Accuracy: {adv_scores[1]}")

    # Analyze how many examples were successfully perturbed
    successful_perturbations = np.sum(original_classes != adv_classes)
    print(f"Successfully perturbed examples: {successful_perturbations} out of {batch_size}")

    # Calculate the robustness as the percentage of successful perturbations
    robustness = successful_perturbations / batch_size
    print(f"Robustness: {robustness}")
    percentage_successful_perturbations = successful_perturbations / batch_size * 100
    print(f"Percentage of successful perturbations: {percentage_successful_perturbations}%")





In [26]:

def robustness_main(nn_type, mfcc, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units):
    # General Confusion Matrix
    confusion_matrix_accumulated = np.zeros((10, 10))
    accuracies = []

    for i in range(1, 11):

        # Define test set paths
        X_test_path = 'UrbanSound8K/audio/fold'+str(i)+'_4sec_mfccs_13/3D_array.npy'
        Y_test_path = 'UrbanSound8K/audio/fold'+str(i)+'_label/3D_array.npy'

        X_val_path1 = 'UrbanSound8K/audio/fold'+str((i+1-1) % 10 + 1)+f'_4sec_mfccs_{mfcc}/3D_array.npy'
        Y_val_path1 = 'UrbanSound8K/audio/fold'+str((i+1-1) % 10 + 1)+'_label/3D_array.npy'
        X_val_path2 = 'UrbanSound8K/audio/fold'+str((i+2-1) % 10 + 1)+f'_4sec_mfccs_{mfcc}/3D_array.npy'
        Y_val_path2 = 'UrbanSound8K/audio/fold'+str((i+2-1) % 10 + 1)+'_label/3D_array.npy'

        # Combine the validation paths
        X_val_paths = [X_val_path1, X_val_path2]
        Y_val_paths = [Y_val_path1, Y_val_path2]

        # # Define training set paths (all remaining folds)
        X_train_paths = ['UrbanSound8K/audio/fold'+str((j-1) % 10 + 1)+f'_4sec_mfccs_{mfcc}/3D_array.npy' for j in range(i+3, i+10)]
        Y_train_paths = ['UrbanSound8K/audio/fold'+str((j-1) % 10 + 1)+'_label/3D_array.npy' for j in range(i+3, i+10)]

        # Define test set paths
        X_test_path = f'UrbanSound8K/audio/fold{i}_4sec_mfccs_{mfcc}/3D_array.npy'
        Y_test_path = f'UrbanSound8K/audio/fold{i}_label/3D_array.npy'

        # Define validation set paths (wrapping around if i+2 > 10)
        X_val_path1 = f'UrbanSound8K/audio/fold{((i+1-1) % 10 + 1)}_4sec_mfccs_{mfcc}/3D_array.npy'
        Y_val_path1 = f'UrbanSound8K/audio/fold{((i+1-1) % 10 + 1)}_label/3D_array.npy'
        X_val_path2 = f'UrbanSound8K/audio/fold{((i+2-1) % 10 + 1)}_4sec_mfccs_{mfcc}/3D_array.npy'
        Y_val_path2 = f'UrbanSound8K/audio/fold{((i+2-1) % 10 + 1)}_label/3D_array.npy'

        # Combine the validation paths
        X_val_paths = [X_val_path1, X_val_path2]
        Y_val_paths = [Y_val_path1, Y_val_path2]

        # Define training set paths (all remaining folds)
        X_train_paths = [f'UrbanSound8K/audio/fold{((j-1) % 10 + 1)}_4sec_mfccs_{mfcc}/3D_array.npy' for j in range(i+3, i+11)]
        Y_train_paths = [f'UrbanSound8K/audio/fold{((j-1) % 10 + 1)}_label/3D_array.npy' for j in range(i+3, i+11)]

        # Load the datasets from the paths
        X_test = np.load(X_test_path)
        Y_test = np.load(Y_test_path)


        X_val1 = np.load(X_val_path1)
        X_val2 = np.load(X_val_path2)
        Y_val1 = np.load(Y_val_path1)
        Y_val2 = np.load(Y_val_path2)

        X_train = [np.load(path) for path in X_train_paths]
        Y_train = [np.load(path) for path in Y_train_paths]


        # Encontrar o tamanho mínimo entre todos os folds
        min_size = min([X_test.shape[0], X_val1.shape[0], X_val2.shape[0]] + [x.shape[0] for x in X_train])

        # Redimensionar os dados de cada fold para o tamanho mínimo
        X_test_resized = X_test[:min_size]
        Y_test_resized = Y_test[:min_size]

        X_val1_resized = X_val1[:min_size]
        Y_val1_resized = Y_val1[:min_size]

        X_val2_resized = X_val2[:min_size]
        Y_val2_resized = Y_val2[:min_size]

        X_train_resized = [x[:min_size] for x in X_train]
        Y_train_resized = [y[:min_size] for y in Y_train]


        # Combine the validation sets
        X_test = X_test_resized
        Y_test = Y_test_resized
        X_val = np.concatenate((X_val1_resized, X_val2_resized), axis=2)
        Y_val = np.concatenate((Y_val1_resized, Y_val2_resized), axis=2)
        X_train = np.concatenate((X_train_resized[0], X_train_resized[1], X_train_resized[2], X_train_resized[3], X_train_resized[4], X_train_resized[5], X_train_resized[6]), axis=2)
        Y_train = np.concatenate((Y_train_resized[0], Y_train_resized[1], Y_train_resized[2], Y_train_resized[3], Y_train_resized[4], Y_train_resized[5], Y_train_resized[6]), axis=2)

        X_test = X_test.transpose(2,1,0)
        Y_test = Y_test.transpose(2,1,0)
        X_val = X_val.transpose(2,1,0)
        Y_val = Y_val.transpose(2,1,0)
        X_train = X_train.transpose(2,1,0)
        Y_train = Y_train.transpose(2,1,0)

    #-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

        clear_session()  # Clear the previous model
        
        # Call de neural network
        input_shape = (751, mfcc)
        num_classes = 10

        #Neural Network Initialization
        if nn_type == "mlp":
            model = build_mlp_model(input_shape, num_classes, l2_lambda, n_hidden_units)

        elif nn_type == "rnn":
            model = build_rnn_model(input_shape, num_classes, l2_lambda, n_hidden_units)

        else:
            print("Introduce a valid neural network")





        model.summary() # print The neural network's architecture

        # Define the optimizer
        optimizer = legacy.SGD(learning_rate=learning_rate, momentum=momentum)

        #output layer
        model.add(layers.Dense(10, activation='softmax'))

        Y_train = np.squeeze(Y_train, axis=1)
        Y_val = np.squeeze(Y_val, axis=1)


        # Compile the model.
        model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )


        history = model.fit(

            #training dataset
            X_train,
            Y_train,

            epochs=n_epochs,  # Number of epochs
            batch_size=batch_size, # Number of samples per batch

            #validation dataset
            validation_data=(X_val, Y_val)
        )

#----------------------------------------------------------------------------------------------------------------- Adding Robustness --------------------------------------------------------------------------------------------------------------------------
       
        # Select a batch of test data for generating adversarial examples
        batch_X_test = X_test[:batch_size]
        batch_Y_test = Y_test[:batch_size]

        print(batch_X_test.shape)
        print(batch_Y_test.shape)

        batch_Y_test = np.squeeze(batch_Y_test, axis=1)

        if batch_Y_test.ndim == 3:
            batch_Y_test = batch_Y_test[:, 0, :]

        print(batch_Y_test.shape)  

        # Generate adversarial examples
        adv_X_test, original_classes, adv_classes = generate_adversarial_examples(batch_X_test, model)

        # Evaluate the model on adversarial examples
        adv_scores = model.evaluate(adv_X_test, batch_Y_test, verbose=0)
        print(f"Adversarial Test Accuracy: {adv_scores[1]}")

        # Analyze how many examples were successfully perturbed
        successful_perturbations = np.sum(original_classes != adv_classes)
        print(f"Successfully perturbed examples: {successful_perturbations} out of {batch_size}")


        # Calculate the robustness as the percentage of successful perturbations
        robustness = successful_perturbations / batch_size
        print(f"Robustness: {robustness}")
        percentage_successful_perturbations = successful_perturbations / batch_size * 100
        print(f"Percentage of successful perturbations: {percentage_successful_perturbations}%")

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

        Y_train_classes = np.squeeze(Y_train).argmax(axis=-1) if Y_train.ndim == 3 else Y_train
        Y_val_classes = np.squeeze(Y_val).argmax(axis=-1) if Y_val.ndim == 3 else Y_val

        # Generate predictions
        Y_pred = model.predict(X_test)  # Predicted probabilities

        # Convert predictions to class labels
        Y_pred_classes = np.argmax(Y_pred, axis=-1)  # Convert probabilities to class labels

        # Flatten the one-hot encoded labels to 1D if they are 3D for Y_test
        Y_test_classes = np.squeeze(Y_test).argmax(axis=-1) if Y_test.ndim == 3 else Y_test

        # Make sure the confusion matrix function is correctly imported
        from sklearn.metrics import confusion_matrix

        # Calculate the confusion matrix
        conf_matrix = confusion_matrix(Y_test_classes, Y_pred_classes)

        # Add the current confusion matrix to the accumulated confusion matrix
        confusion_matrix_accumulated += conf_matrix

        # Evaluate the model, ensuring the Y_test used here matches the format expected by the model
        scores = model.evaluate(X_test, np.squeeze(Y_test), verbose=0)
        print(f"Test accuracy for fold {i}:", scores[1])


        

        # Calculate precision and recall
        precision = precision_score(Y_test_classes, Y_pred_classes, average='macro')
        recall = recall_score(Y_test_classes, Y_pred_classes, average='macro')

        print(f"Precision: {precision}")
        print(f"Recall: {recall}")

        accuracies.append(scores[1])
        print(f"Test accuracy for fold {i}:", scores[1])



## Test for a single iteration

### For 13 coefficients

### For 13 coefficients

In [27]:
# The best Hyperparameters given by tensorboard - The best score: 0.26345932483673096
n_epochs = 10
batch_size = 32
learning_rate = 0.0001
momentum = 0.8
l2_lambda = 0.0001
n_hidden_units = 64


In [28]:
robustness_test("mlp", 13, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9763)              0         
                                                                 
 dense (Dense)               (None, 64)                624896    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 10)                6

#### Recurrent Neural Network

In [29]:
robustness_test("bd_rnn", 13, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 751, 128)          39936     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
Total params: 140042 (547.04 KB)
Trainable params: 140042 (547.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
189/189 [==============================] - 213s 1s/step - loss: 2.3341 - accuracy: 0.1193 - val_loss: 2.3283 - val_accuracy: 0.1131
Epoch

### For 25 coefficients

####  Multilayer Perceptron

In [30]:
# The best Hyperparameters given by tensorboard - The best score: 0.26345932483673096
n_epochs = 5
batch_size = 32
learning_rate = 0.0005
momentum = 0.9
l2_lambda = 0.0001
n_hidden_units = 64


#### Recurrent Neural Network

In [31]:
robustness_test("mlp", 25, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 18775)             0         
                                                                 
 dense (Dense)               (None, 64)                1201664   
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 10)                6

In [32]:
robustness_test("bd_rnn", 25, n_epochs, batch_size, learning_rate, momentum, l2_lambda, n_hidden_units)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 751, 128)          46080     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
Total params: 146186 (571.04 KB)
Trainable params: 146186 (571.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
189/189 [==============================] - 208s 1s/step - loss: 2.3385 - accuracy: 0.1132 - val_loss: 2.3256 - val_accuracy: 0.0965
Epoch 